In [ ]:
!pip install pandas xlsxwriter

In [77]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


25/02/28 13:25:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [78]:
 spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_1000_50_product")


DataFrame[]

In [79]:
import os
iceberg_table_dir = "../warehouse/nyc/taxis_1000_50_product"
metadata_dir = f"{iceberg_table_dir}/metadata"
data_dir = f"{iceberg_table_dir}/data"
input_data_dir = f"../input_data"
analysis_info = []
records_before_op = 0

def append_to_file(file_path, msg):
    open_mode = "a"
    if not os.path.exists(file_path):
        open_mode = "w"

    # Open the CSV file in write mode
    with open(file_path, open_mode) as file:
        writer = csv.writer(file)
        
        if open_mode=="w":
            #writing header of the columns
            writer.writerows([list(msg.keys())])    

        row_values = [list(msg.values())]
        # Write the data to the CSV file
        writer.writerows(row_values)

def get_size():
    # List the metadata files
    manifest_pattern = re.compile(r".*-m\d+\.avro$")
    metadata_files = os.listdir(metadata_dir)
    
    # Initialize variables to store the sizes of different types of metadata files
    snap_avro_size = 0
    metadata_json_size = 0
    m_avro_size = 0

    data_dir_size = 0
    # get data dir size
    data_dir_files = os.listdir(data_dir)
    # print(data_dir_files)
    for filename in data_dir_files:
        file_path = os.path.join(data_dir, filename)
        data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
    # Iterate through the metadata files and calculate their sizes
    for file in metadata_files:
        file_path = os.path.join(metadata_dir, file)
        file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
        if file.startswith("snap-") and file.endswith(".avro"):
            snap_avro_size += file_size_kb
        elif file.endswith(".metadata.json"):
            metadata_json_size += file_size_kb
        elif manifest_pattern.match(file):
            m_avro_size += file_size_kb
    
    # Print the time taken and the sizes of the metadata files
    # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
    # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
    # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
    # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

    return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [85]:
import random
import string
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, DateType
)
from datetime import datetime, timedelta

# Initialize Spark session
spark = SparkSession.builder.appName("Generate Taxis Data").getOrCreate()

def generate_random_month_date():
    """Generate a random date with the first day of a month."""
    start_date = datetime(2020, 1, 1)
    end_date = datetime.today()
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    random_date = start_date + timedelta(days=random_days)
    
    return random_date.replace(day=1)  # Ensuring DATE format

# Define schema ensuring extra_col_3 is DateType
schema = StructType([
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 0 else  
    StructField(f"extra_col_{i}", IntegerType(), True) if i % 4 == 1 else  
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 2 else  
    StructField(f"extra_col_{i}", DateType(), True)  # Ensure DATE type for extra_col_3
    for i in range(50)
])

def generate_records(n):
    """Generate records based on schema."""
    data = []
    for _ in range(n):
        row = [
            ''.join(random.choice(string.ascii_uppercase) for _ in range(10)) if i % 4 == 0 else
            random.randint(1, 10000) if i % 4 == 1 else
            random.choice(["CAR", "TW", "CV"]) if i % 4 == 2 else
            generate_random_month_date()  # Ensure DATE type
            for i in range(50)
        ]
        data.append(row)
    
    return data

# Generate Data
records = generate_records(1000)  # Adjust number of records

# Create DataFrame
df = spark.createDataFrame(records, schema)

# Ensure extra_col_3 is stored as DateType
df = df.withColumn("extra_col_3", F.to_date(F.col("extra_col_3")))

# Show sample records
df.show(5)

# Save to Iceberg Table
# df.writeTo("demo.nyc.taxis_1000_50_product").create()
df.writeTo("demo.nyc.taxis_1000_50_product") \
    .partitionedBy("extra_col_2") \
    .create()



25/02/28 13:27:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|extra_col_0|extra_col_1|extra_col_2|extra_col_3|extra_col_4|extra_col_5|extra_col_6|extra_col_7|extra_col_8|extra_col_9|extra_col_10|extra_col_11|extra_col_12|extra_col_13|extra_col_14|extra_col_15|extra_col_16|extra_col_17|extra_col_18|extra_col_19|extra_col_20|extra_col_21|extra_col_22|extra_col_23|extra_col_24|extra_col_25|extra_col_26|extra_col_27|ext

In [86]:
# df = spark.table("demo.nyc.taxis_1000_50_product")
# df.show()

In [87]:
import time
import os
from pyspark.sql import functions as F

input_data_dir = "../input_data"
output_dir = "../output"

file_type = input("Enter input file type (csv or parquet): ").lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

df = spark.table("demo.nyc.taxis_1000_50_product")
records_before_op = df.count()

for file in input_files:
    print(f"Processing file: {file}")
    file_path = os.path.join(input_data_dir, file)

    start_time = time.time()
    
    # Read Data
    if file_type == "parquet":
        df = spark.read.parquet(file_path)
    else:
        df = spark.read.csv(file_path, header=True)
        df = df.select(
            *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(50)]
        )
    
    # Ensure extra_col_3 is in DATE format
    df = df.withColumn("extra_col_3", F.to_date(F.col("extra_col_3")))

    # Insert into Iceberg
    df.writeTo("demo.nyc.taxis_1000_50_product").append()

    print(f"Inserted {df.count()} records in {time.time() - start_time:.2f} sec")


Enter input file type (csv or parquet):  parquet


Processing file: records_1000_1_1740744671.parquet
Inserted 1000 records in 1.40 sec


In [88]:
import time

# Start the timer
start_time = time.time()

# Define the complex SQL query for 'nyc.taxis_10000_50COLUMNS'
query = """
SELECT
extra_col_2 , extra_col_1
FROM demo.nyc.taxis_1000_50_product
where extra_col_2 = 'CAR'
"""

# Execute the SQL query
df = spark.sql(query)

# Stop the timer
end_time = time.time()

# Show the result
df.show(truncate=False)

# Print the time taken for the query
print(f"Time taken for the query: {end_time - start_time} seconds")


+-----------+-----------+
|extra_col_2|extra_col_1|
+-----------+-----------+
|CAR        |2415       |
|CAR        |2372       |
|CAR        |6793       |
|CAR        |4729       |
|CAR        |1181       |
|CAR        |1615       |
|CAR        |866        |
|CAR        |4326       |
|CAR        |6330       |
|CAR        |7213       |
|CAR        |8271       |
|CAR        |9111       |
|CAR        |9215       |
|CAR        |3581       |
|CAR        |4407       |
|CAR        |1463       |
|CAR        |7519       |
|CAR        |1892       |
|CAR        |4969       |
|CAR        |4185       |
+-----------+-----------+
only showing top 20 rows

Time taken for the query: 0.014000415802001953 seconds


In [92]:
# df = spark.sql("SELECT * FROM demo.nyc.taxis_1000_50_product.files")
# df.show()

In [89]:
# df=spark.sql("SELECT * FROM demo.nyc.taxis_1000_50_product.partitions")
# df.show()

In [76]:
# df = spark.sql("CREATE TABLE demo.nyc.taxis_1000_50_product_partitioned USING ICEBERG PARTITIONED BY (extra_col_2)")


In [93]:
# spark.sql("DESCRIBE FORMATTED demo.nyc.taxis_1000_50_product").show(truncate=False)


In [90]:
# spark.sql("SELECT DISTINCT extra_col_2 FROM demo.nyc.taxis_1000_50_product_partitioned").show(truncate=False)


In [91]:
# 